In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"
import torchvision 
from torchvision import datasets,transforms
# torchvision内置了常见数据集和最常用的模型

C:\Users\22789\Anaconda3\envs\pytorch_py\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
transformation = transforms.Compose([
    transforms.ToTensor()
])

In [3]:
train_dataset = datasets.MNIST(
    'E:\\python_project\\data/',
    train = True,
    transform= transformation,
    download =True
)

In [4]:
test_dataset = datasets.MNIST(
    'E:\\python_project\\data/',
    train = False,
    transform= transformation,
    download =True
)

In [5]:
batch =32
train_dl = torch.utils.data.DataLoader(train_dataset,batch_size =batch,shuffle =True)
test_dl = torch.utils.data.DataLoader(test_dataset,batch_size=batch)

In [6]:
x,y =next(iter(train_dl))
print(x.shape)

torch.Size([32, 1, 28, 28])


# 1.创建CNN卷积网络模型

In [7]:
class Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1,6,5)
        self.pool = nn.MaxPool2d((2,2)) #图像原有的长和宽变为二分之一
        self.conv2 = nn.Conv2d(6,16,5)
        self.linear_1 = nn.Linear(16*4*4,256)
        self.linear_2 = nn.Linear(256,10)
    def forward(self,input):
        x = F.relu(self.conv1(input))
        x = self.pool(x)
        x = F.relu(self.conv2(x))
        x = self.pool(x)
        x = x.view(x.size(0),-1)
        x = self.linear_1(x)
        x = self.linear_2(x)
        return x
        

In [8]:
torch.cuda.is_available()

True

In [9]:
torch.__version__

'1.8.0+cu111'

In [10]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [11]:
model = Model()
model = model.to(device)

In [12]:
loss_fn = torch.nn.CrossEntropyLoss()
opt = optim.Adam(model.parameters(),lr=0.0001)

In [13]:
def fit(epoch,model,trainloader,testloader):
    correct = 0
    total = 0 
    running_loss = 0
    for x,y in trainloader:
        x,y = x.to(device),y.to(device)
        y_pred = model(x)
        loss =loss_fn(y_pred,y)
        opt.zero_grad()
        loss.backward()
        opt.step()
        with torch.no_grad():
            y_pred = torch.argmax(y_pred,dim=1)
            correct += (y_pred == y).sum().item()
            total += y.size(0)
            running_loss += loss.item()
    epoch_loss = running_loss
    epoch_acc = correct/total 
    
    test_correct = 0
    test_total = 0
    test_running_loss = 0
    with torch.no_grad():
        for x,y in testloader:
            x,y = x.to(device),y.to(device)
            y_pred = model(x)
            loss = loss_fn(y_pred,y)
            y_pred = torch.argmax(y_pred,dim=1)
            test_correct += (y_pred == y).sum().item()
            test_total +=y.size(0)
            test_running_loss +=loss.item()
    test_loss = test_running_loss
    test_acc = test_correct/test_total
    print('epoch: ', epoch+1, 
          'loss： ', round(epoch_loss, 3),
          'accuracy:', round(epoch_acc, 3),
          'test_loss： ', round(test_loss, 3),
          'test_accuracy:', round(test_acc, 3)
             )

In [14]:
for epoch in range(20):
    fit(epoch,model,train_dl,test_dl)

epoch:  1 loss：  1009.34 accuracy: 0.859 test_loss：  56.634 test_accuracy: 0.945
epoch:  2 loss：  304.352 accuracy: 0.952 test_loss：  37.789 test_accuracy: 0.963


KeyboardInterrupt: 

In [ ]:
epochs = 20
for epoch in range(epochs):
    total = 0
    correct = 0
    running_loss = 0
    for x,y in train_dl:
        y_pred = model(x)
        loss = loss_fn(y_pred,y)
        opt.zero_grad()
        loss.backward()
        opt.step()
        with torch.no_grad():
            y_pred = torch.argmax(y_pred,dim=1)
            correct += (y_pred==y).sum().item()
            total += y.size(0) #统计每个批次的标签数量和
            running_loss += loss.item()
    print(f'第{epoch+1}轮训练完成，loss:{running_loss},acc:{correct/total}')
    
    # 测试集准确度验证
    test_correct = 0
    test_total = 0
    test_loss = 0
    with torch.no_grad():
        for x,y in test_dl:
            y_pred = model(x)
            loss = loss_fn(y_pred,y)
            y_pred = torch.argmax(y_pred,dim=1)
            test_correct += (y_pred==y).sum().item() 
            test_total += y.size(0)
            test_loss += loss.item()
        print(f"测试集acc：{test_correct/test_total},loss:{test_loss}")